- This was done using gensim with lemmatization and stopwords still included as that was found to be the best method
- Trained Gensim model over time period binning as descriped in RegenX paper and saves model for further analysis

# Training Gensim model on neuroscience papers

In [ ]:
!pip install python-docx

In [ ]:
!pip install glove_python-binary

In [ ]:
from docx import Document
import nltk
nltk.download('punkt')
import re
from nltk import sent_tokenize
import pandas as pd
from nltk.corpus import stopwords
nltk.download('stopwords')
import pickle
import numpy as np
import glob

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from nltk.stem import WordNetLemmatizer
import nltk 
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
!git clone 'https://github.com/igorbrigadir/stopwords.git'

fatal: destination path 'stopwords' already exists and is not an empty directory.


In [ ]:
alir3z4_data = '/content/stopwords/en/alir3z4.txt'

more_stops = pd.read_csv('/content/stopwords/en/alir3z4.txt')
new_stops = list(more_stops["'ll"])

In [ ]:
DOMAIN_STOPS = {'pubmed', 'et', 'al', 'page'}
STOPWORDS =  set(stopwords.words('english') + stopwords.words('german') +  stopwords.words('dutch') + stopwords.words('french') +  stopwords.words('spanish')  + new_stops) | DOMAIN_STOPS
STOPWORDS = set(STOPWORDS)

In [ ]:
len(STOPWORDS)

2011

In [ ]:
'a' in STOPWORDS

True

In [ ]:
ROOT = "/content/drive/MyDrive/regen_x"

In [ ]:
def get_docx(file_path):
    doc = []
    for para in Document(file_path).paragraphs:
        if para.text == "":
            continue
        doc += (sent_tokenize(para.text.lower())) # we lower text here
    return doc


def get_start_stop():
    domain_stops = {'pubmed', 'et', 'al', 'page'}
    with open('/content/stopwords/en/alir3z4.txt', 'r') as fn:
        new_stops = [line.strip() for line in fn.readlines()]
    STOPWORDS =  set(stopwords.words('english') + stopwords.words('german') +  stopwords.words('dutch') + stopwords.words('french') +  stopwords.words('spanish')  + new_stops) | domain_stops

    fn = glob.glob(ROOT + '/data/start-words/*')
    ALL_STARTS = [pickle.load(open(f , 'rb')) for f in fn]
    STARTWORDS = {}
    for f in ALL_STARTS:
      STARTWORDS.update(f)
    STARTWORDS = set(STARTWORDS.keys())

    assert(type(STOPWORDS)==set and type(STARTWORDS)==set)
    return (STARTWORDS, STOPWORDS)

In [ ]:
STARTWORDS, STOPWORDS = get_start_stop()

In [ ]:
from gensim.models import Word2Vec

# Optimizing Training Function

You **don't** want to do incremental training for the reasons given in [this answer](https://stackoverflow.com/questions/42746007/incremental-word2vec-model-training-in-gensim)

# Time Period Binning

In [ ]:
from natsort import natsorted
import os

In [ ]:
ROOT = "/content/drive/MyDrive/regen_x"
NUM_BINS = 16

In [ ]:
def get_time_per_list(NUM_BINS):
    all_paths = natsorted(glob.glob(ROOT + '/data/ocr_paper_COMPREHENSIVE/*/'))
    all_path_chunked = np.array_split(all_paths , NUM_BINS)
    all_paths = np.array_split(all_paths , NUM_BINS)
    time_periods = {}
    time_per_list = []
    for i , file_chunk in enumerate(all_paths):
        time_periods[i] = file_chunk
        for j in range(len(time_periods[i])):
            time_periods[i][j] = time_periods[i][j].split('/')[-2]
        time_periods[i] = str(time_periods[i][0]) + '-' +  str(time_periods[i][-1])
        time_per_list.append(time_periods[i])
    return (time_per_list, all_path_chunked)

In [ ]:
(time_per_list, all_path_chunked) = get_time_per_list(NUM_BINS)

In [ ]:
all_path_chunked[0]

array(['/content/drive/MyDrive/regen_x/data/ocr_paper_COMPREHENSIVE/1776/',
       '/content/drive/MyDrive/regen_x/data/ocr_paper_COMPREHENSIVE/1795/',
       '/content/drive/MyDrive/regen_x/data/ocr_paper_COMPREHENSIVE/1820/',
       '/content/drive/MyDrive/regen_x/data/ocr_paper_COMPREHENSIVE/1824/',
       '/content/drive/MyDrive/regen_x/data/ocr_paper_COMPREHENSIVE/1826/',
       '/content/drive/MyDrive/regen_x/data/ocr_paper_COMPREHENSIVE/1827/',
       '/content/drive/MyDrive/regen_x/data/ocr_paper_COMPREHENSIVE/1828/',
       '/content/drive/MyDrive/regen_x/data/ocr_paper_COMPREHENSIVE/1831/',
       '/content/drive/MyDrive/regen_x/data/ocr_paper_COMPREHENSIVE/1832/'],
      dtype='<U65')

In [ ]:
# For gensim we need to split the document into sentences. 
# Then concatenate all the documents of a given year into one big array with all their sentences. 

# This functions takes a folder of files and returns one array with 
# all of the files processed sentences(which themselves are a list of words) as elements 
# def get_proc_docs(year_path, STARTWORDS, STOPWORDS, max_papers=None, verbose=True):
#   file_paths = glob.glob(year_path + "*.docx")

#   print("Number of files: {}".format(len(file_paths)))
#   if len(file_paths) == 0:
#     # raise Exception("Folder has no files - maybe drive was not mounted?")
#     pass 
#   ## -- Collecting Papers from Given Year -- ##
#   proc_docs = [] 

#   counter = 1
#   length = len(file_paths)
#   for f in file_paths:
#     # gives it to us in sentences! 
#     doc = get_docx(f)
#     # proc_doc = [word for word in re.findall(r'\w+', doc.lower()) if ((word in STARTWORDS) and (len(word) > 2) and (word not in STOPWORDS))]

#     for sentence in doc:
#       # don't think we need to remove stopwords and such if we're training embeddings 
#       # do lemmatization here as well 
#       proc_sentence = [word for word in re.findall(r'\w+', sentence)]
#       proc_sentence = do_lemmatizing(proc_sentence) 
#       proc_docs.append(proc_sentence)  

#     if(verbose):
#       print("\t{}/{}".format(counter, length))
#     counter += 1

#     if max_papers != None:
#       if counter == max_papers+1:
#         break 

#   return proc_docs

In [ ]:
# for lemmatization 
import spacy
# Initialize spacy 'en' model, keeping only tagger component needed for lemmatization
nlp = spacy.load('en', disable=['parser', 'ner'])

In [ ]:
def do_stemming(filtered):
	stemmed = []
	for f in filtered:
		stemmed.append(PorterStemmer().stem(f))
		#stemmed.append(LancasterStemmer().stem(f))
		#stemmed.append(SnowballStemmer('english').stem(f))
	return stemmed

def do_lemmatizing(filtered):
  # convert list to string 
  spacy_parsed_text = nlp(" ".join(filtered)) 
  # Get the lemma for each token in the parsed text 
  
  # I wanted to keep pronouns so not taking lemma if it's a pronoun but if you want to remove pronouns use below commented line 
  # return " ".join([token.lemma_ for token in doc])

  # return as list of words again 
  return [token.lemma_ if token.lemma_ != '-PRON-' else token.lower_ for token in spacy_parsed_text]

In [ ]:
def get_proc_docs(year_path, STARTWORDS, STOPWORDS, max_papers=None, verbose=True, use_porter=False, useStopWords=False):
  file_paths = glob.glob(year_path + "*.docx")

  print("Number of files: {}".format(len(file_paths)))
  if len(file_paths) == 0:
    # raise Exception("Folder has no files - maybe drive was not mounted?")
    pass 
  ## -- Collecting Papers from Given Year -- ##
  proc_docs = [] 

  counter = 1
  length = len(file_paths)
  for f in file_paths:
    doc = get_docx(f)
    
    for sentence in doc:
      # don't think we need to remove stopwords and such if we're training embeddings 
      # do lemmatization here as well 

      proc_sentence = [] 
      if useStopWords:
        proc_sentence = [word for word in re.findall(r'\w+', sentence) if ((len(word) > 2) and (word not in STOPWORDS))]
      else:
        proc_sentence = [word for word in re.findall(r'\w+', sentence)]

      if use_porter:
        proc_sentence = do_stemming(proc_sentence) 
      else:
        proc_sentence = do_lemmatizing(proc_sentence) 
      
      proc_docs.append(proc_sentence)  

    if(verbose):
      print("\t{}/{}".format(counter, length))
    counter += 1

    if max_papers != None:
      if counter == max_papers+1:
        break 

  return proc_docs

In [ ]:
MODEL_PATH = "/content/drive/MyDrive/Colab Notebooks/NLP - Lab/WordEmbeddings/Models/Gensim_Lemmatized_Without_Stopwords/"

if not os.path.exists(MODEL_PATH):
    os.makedirs(MODEL_PATH)

In [ ]:
### Train and Save Models for all time periods ##

counter = 0 
for time_period_paths in all_path_chunked:
  cur_time_period = time_per_list[counter] 
  print("Current Time Period: {}".format(cur_time_period))
  all_proc_docs_time_period = [] 

  for i, year_path in enumerate(time_period_paths):
    print("{}/{}".format(i+1, len(time_period_paths)))

    proc_doc_cur_year = get_proc_docs(year_path, STARTWORDS, STOPWORDS, max_papers=None, verbose=True, use_porter=False, useStopWords=True) 

    all_proc_docs_time_period += all_proc_docs_time_period + proc_doc_cur_year 

  # Train the embeddings! 
  print("Training word embeddings for {}...".format(cur_time_period))
  model = Word2Vec(sentences=all_proc_docs_time_period, min_count=1) 

  # Sanity check 
  print(model.wv.most_similar("eye", topn=10))

  # Store just the words + their trained embeddings.
  word_vectors = model.wv
  word_vectors.save(MODEL_PATH + "{}.wordvectors".format(cur_time_period))

  # Manually create space 
  del all_proc_docs_time_period
  del word_vectors 
  del model 

  counter += 1

Current Time Period: 1776-1832
1/9
Number of files: 1
	1/1
2/9
Number of files: 1
	1/1
3/9
Number of files: 1
	1/1
4/9
Number of files: 2
	1/2
	2/2
5/9
Number of files: 1
	1/1
6/9
Number of files: 1
	1/1
7/9
Number of files: 2
	1/2
	2/2
8/9
Number of files: 1
	1/1
9/9
Number of files: 2
	1/2
	2/2
Training word embeddings for 1776-1832...
[('person', 0.7005709409713745), ('correspondence', 0.6530425548553467), ('situate', 0.6521275043487549), ('extract', 0.6384673714637756), ('object', 0.6352414488792419), ('squinting', 0.6308776140213013), ('tumour', 0.6193944215774536), ('ponde', 0.6065702438354492), ('partially', 0.602207362651825), ('crecum', 0.6004467010498047)]
Current Time Period: 1835-1846
1/9
Number of files: 2
	1/2
	2/2
2/9
Number of files: 1
	1/1
3/9
Number of files: 0
4/9
Number of files: 1
	1/1
5/9
Number of files: 1
	1/1
6/9
Number of files: 1
	1/1
7/9
Number of files: 1
	1/1
8/9
Number of files: 1
	1/1
9/9
Number of files: 2
	1/2
	2/2
Training word embeddings for 1835-184

In [ ]:
# Exceution of above function took: 59 mins 19 seconds